In [ ]:
import pandas as pd 
import spotipy 
import spotipy.util as sp_util
from spotipy.oauth2 import SpotifyClientCredentials 
import seaborn as sb
from dotenv import load_dotenv
from os.path import join, dirname
import os

%matplotlib inline
dotenv_path = '.env'
load_dotenv(dotenv_path)

cid = os.environ.get("cid")
secret = os.environ.get("secret")
user_id= os.environ.get("user_id")

In [ ]:
sp = spotipy.Spotify() 
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret,) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 
sp.trace=False 

Leo el archivo generado con los obtención desde spotify y elimino la columna del índice

In [ ]:
df = pd.read_csv('data/audio_features3.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

Separo el nombre del artista y el nombre de la canción en dos columnas nuevas

In [ ]:
df['artist'], df['song'] = df['name'].str.split('|', 1).str
df.drop('name', axis=1, inplace=True)


Trabajo sobre los artistas para idetificarlos y obtener las canciones de cada artista desde spotify

In [ ]:
df_aux = df[df['artist'] != '']
artist = df_aux.artist.unique()
print('cantidad artistas: {}'.format(len(artist)))

Busco canciones de cada artista encontrado

In [ ]:
df_artist = pd.DataFrame({},columns=['id', 'artist', 'name'])

for i in artist:
    results = sp.search(q=i, limit=40)
    for t in results['tracks']['items']:
        aux = pd.DataFrame(data=[[t['id'], i, t['name'], t['duration_ms'], t['popularity'], t['explicit']]], columns=['id', 'artist', 'name', 'duration','popularity','explicit'])
        df_artist = df_artist.append(aux)

len(df_artist)

In [ ]:
df_artist.set_index('id', inplace=True)
df_artist.to_csv('data/songs_artist_name.csv',sep=';')

Ahora que tengo 14mil canciones obtengo sus feature

In [ ]:
#acepta un array con ids
df_artist = pd.read_csv('data/songs_artist_name.csv',sep=';')
#df_artist.drop('Unnamed: 0',axis=1, inplace=True)
#df_artist.set_index('id', inplace=True)
df_artist.head()
#sp.audio_features(ids)

In [ ]:
id = '7rEstJLdWhDSoh84cj5bqb'

feat = sp.audio_features(tracks=id)

feat
#df_artist.index.values.tolist()